# FAKE_NEWS_DETECTION(NLP) 

In [ ]:
import pandas as pd 

In [ ]:
data=pd.read_csv('news.csv')

In [ ]:
data.head(10)

In [ ]:
data.shape

## Cleaning data

In [ ]:
# now we can see Unnamed: 0 column that is no use 
# and droping title that is just heading of text

In [ ]:
data.drop(['Unnamed: 0','title'],axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
# we can now check for Nan value

In [ ]:
data.isnull().sum()

In [ ]:
# now changing label as it is Categorical data we have to change it to numeric value(0,1).
# we can change it by different encoder technique like labelencoder or one hot encoder but,binary 
# this can be done with replace method also.

In [ ]:
data.label.replace(('FAKE','REAL'),(0,1),inplace=True)

In [ ]:
data.head()

In [ ]:
X=data.drop('label',axis=1)
Y=data['label']

### visualization

In [ ]:
import seaborn as sns 
sns.countplot(x='label',data=data)

###### label is evenly distributed good to go ahead

In [ ]:
# now using Natural language processing

In [ ]:
import re  # regular pattern of regualr expression are created and we can escape some data which is not needed

In [ ]:
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()
corpus = []
for i in range(len(data)):
    review=re.sub('[^a-z-A-Z]', ' ',data['text'][i])
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review= ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v=TfidfVectorizer(max_features=5000,ngram_range=(1,3))
X= tfidf_v.fit_transform(corpus).toarray()

In [ ]:
X

### splitting dataset by train_test_split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.25,random_state=0)

In [ ]:
Y_test.head()

tfidf_v.get_feature_names()[:20]

In [ ]:
tfidf_v.get_params()

In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
count_data=pd.DataFrame(X_train, columns=tfidf_v.get_feature_names())

In [ ]:
count_data.head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False,
                         title='Confusion matrix', 
                         cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks=np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
        
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Multinomial Naive-Bayes Algorithm

In [ ]:
from sklearn.naive_bayes import MultinomialNB
classifier=MultinomialNB()

In [ ]:
import numpy as np
import itertools
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
classifier.fit(X_train,Y_train)
pred=classifier.predict(X_test)
score=accuracy_score(Y_test, pred)*100
print("accuracy:   %0.3f"% score)
cm=confusion_matrix(Y_test, pred)
plot_confusion_matrix(cm, classes=["FAKE","REAL"])

In [ ]:
# Multinomial classifier with hyperparameter(HYPERTUNING) of naive bayes

In [ ]:
classifier=MultinomialNB(alpha=0.1)

In [ ]:
previous_score=0
for alpha in np.arange(0,1,0.1):
    sub_classifier=MultinomialNB(alpha=alpha)
    sub_classifier.fit(X_train,Y_train)
    y_pred=sub_classifier.predict(X_test)
    score=accuracy_score(Y_test,y_pred)
    if score>previous_score:
        classifier=sub_classifier
    print("Alpha: {},Score: {}".format(alpha,score))

In [ ]:
# for alpha=0 applying naive-bayes

In [ ]:
classifier=MultinomialNB(alpha=0)
classifier.fit(X_train,Y_train)
pred=classifier.predict(X_test)
score=accuracy_score(Y_test, pred)*100
print("accuracy:   %0.3f"% score)
cm=confusion_matrix(Y_test, pred)
plot_confusion_matrix(cm, classes=["FAKE","REAL"])

## Passive aggreesive classifier algorithm

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier
linear_clf = PassiveAggressiveClassifier(n_iter_no_change=50)

In [ ]:
linear_clf.fit(X_train,Y_train)
pred = linear_clf.predict(X_test)
score=accuracy_score(Y_test, pred)*100
print("accuracy:   %0.3f"% score)
cm=confusion_matrix(Y_test, pred)
plot_confusion_matrix(cm, classes=["FAKE DATA","REAL DATA"])

In [ ]:
feature_names=tfidf_v.get_feature_names()

In [ ]:
#most fake
sorted(zip(classifier.coef_[0],feature_names))[:20]

In [ ]:
#save_model
import pickle
filename= 'fake_news.pkl'
pickle.dump(linear_clf, open(filename, 'wb'))